In [1]:
import numpy as np

### Optical Simulatio LUT Tree Converter
### ====================================

# Units: pos[mm], mom[eV/c] and c = 1

# LUT format
    # TFile OptSim_LUT_ArgonCube2x2.root
        # TTree PhotonLibraryData
            # Int_t "Voxel"
            # Int_t "OpChannel"
            # Float_t "Visibility"
        # TVectorT<double> Min
        # TVectorT<double> Max
        # TVectorT<double> NDividsions
        
# LUT converter: create_LUT.C

# LUT converter input file: OptSim_LUT_voxel_table.txt

# User input section ===>>>

# Geometry GDML:
# Origin of GDML (x,y,z):
GDML_orig = np.array([0, 0, 0])

# Geometry volLAr:
# Origin of volume (x,y,z):
LAr_orig = np.array([5.845, 0, 0])
# x-, y- and z-dimension of volume:
LAr_dim = np.array([151.362, 630.936, 310.150])*2
# x-, y- and z-coordinates of volume:
LAr_vol = np.array([(LAr_orig-LAr_dim/2),(LAr_orig+LAr_dim/2)])

# Geometry LUT voxels:
# Origin of voxels (x,y,z):
Vox_orig = LAr_vol[0]
# Number of Voxels
Vox_n = np.ceil(LAr_dim/10)
# Make sure to have even numbers of voxels (symmetry)
Vox_n = Vox_n + np.mod(Vox_n,np.full(3,2))

# x-, y- and z-size of LUT voxels
Vox_dim = LAr_dim/Vox_n

# make use of symmetry
Vox_n[1] = Vox_n[1]/2
Vox_n[2] = Vox_n[2]/2

# Event structure:
# Number events:
n_evt = (1E+3)
# Number of photons per event:
n_ph = (1E+4)

# <<< === End of user input section

# NOTES:
# CPU computing time 1000 evts * 10'000 photons (1 thread): ~ 12 min, output-file ~ 500 MB
# 1 (1/4) TPC (make use of symmetries) = 32 (32) * 128 (64) * 64 (32) voxels (10 * 10 mm^2) = 262'144 (65'536) voxels tot.
# Total CPU computing time (single-threaded): ~ 546 d
# Total size of output trees: ~ 33 T

In [2]:
# Generate LUT converter input file:
with open("OptSim_LUT_voxel_table.txt", "w") as text_file:
    VoXYZ = []

    print("{0}\n".format("Opt. vol. orig. (x,y,z) [mm]: %.3f %.3f %.3f" % (LAr_orig[0],LAr_orig[1],LAr_orig[2])))
    print("{0}\n".format("Opt. vol. min (x,y,z) [mm]: %.3f %.3f %.3f" % (LAr_vol[0][0],LAr_vol[0][1],LAr_vol[0][2])))
    print("{0}\n".format("Opt. vol. max (x,y,z) [mm]: %.3f %.3f %.3f" % (LAr_vol[1][0],LAr_vol[1][1],LAr_vol[1][2])))
    print("{0}\n".format("Vox. dim. (x,y,z) [mm]: %.3f %.3f %.3f" % (Vox_dim[0],Vox_dim[1],Vox_dim[2])))
    print("{0}\n".format("Number of voxels (x,y,z) [-]: %d %d %d" % (Vox_n[0],Vox_n[1],Vox_n[2])))
    #print("{0}\n".format("Vox. center coordinates (x,y,z) [mm]:"))
    text_file.write("{0}\n".format("%.3f %.3f %.3f" % (LAr_vol[0][0],LAr_vol[0][1],LAr_vol[0][2])))
    text_file.write("{0}\n".format("%.3f %.3f %.3f" % (LAr_vol[1][0],LAr_vol[1][1],LAr_vol[1][2])))
    text_file.write("{0}\n".format("%.3f %.3f %.3f" % (Vox_dim[0],Vox_dim[1],Vox_dim[2])))
    text_file.write("{0}\n".format("%d %d %d" % (Vox_n[0],Vox_n[1],Vox_n[2])))
    text_file.write("{0}\n".format("%d %d" % (n_evt, n_ph)))
    
    # Voxel center coordinates
    VoxID = 0
    count = 0
    for i in range(int(Vox_n[2])):
        for j in range(int(Vox_n[1])):
            for k in range(int(Vox_n[0])):
                #print(count)
                VoXYZ.append(LAr_vol[0]+np.array([k,j,i])*Vox_dim+Vox_dim/2)
                VoxID = int(k+j*Vox_n[0]+i*Vox_n[0]*Vox_n[1])
                text_file.write("%d %f %f %f\n" % (VoxID, VoXYZ[-1][0],VoXYZ[-1][1],VoXYZ[-1][2]))
                count += 1
                #VoxID3D = (np.floor((VoXYZ[-1]-LAr_vol[0])/Vox_dim))
                #VoxID = int(VoxID3D[0]+VoxID3D[1]*Vox_n[0]+VoxID3D[2]*Vox_n[0]*Vox_n[1])
                #print(VoxID)

Opt. vol. orig. (x,y,z) [mm]: 5.845 0.000 0.000

Opt. vol. min (x,y,z) [mm]: -145.517 -630.936 -310.150

Opt. vol. max (x,y,z) [mm]: 157.207 630.936 310.150

Vox. dim. (x,y,z) [mm]: 9.460 9.858 9.692

Number of voxels (x,y,z) [-]: 32 64 32



In [64]:
import uproot
import pandas as pd

root_filename = 'OptSim_LUT_ArgonCube2x2_test.root'
root_file = uproot.open(root_filename)
root_file.keys()

[b'PhotonLibraryData;1', b'Min;1', b'Max;1', b'NDivisions;1']

In [65]:
tree = root_file['PhotonLibraryData']
branch = tree.arrays()
#branches = tree.arrays(namedecode='utf-8')
branch.keys()

dict_keys([b'Voxel', b'OpChannel', b'Visibility', b'VolID', b'VolIdx', b'YPos', b'Time'])

In [66]:
#df = tree.pandas.df(flatten=False)
df = tree.pandas.df()
df.head()

,Voxel,OpChannel,Visibility,VolID,VolIdx,YPos,Time
entry,,,,,,,
0,0,0,0.008876,0,20,-620.299988,-1.0
1,0,1,0.001358,1,20,-564.900513,-1.0
2,0,2,0.000618,100,20,-499.687347,-1.0
3,0,3,0.000340,101,20,-436.699036,-1.0
4,0,4,0.000247,200,20,-391.563446,-1.0


In [67]:
import plotly.express as px
import numpy as np

fig = px.scatter(df,
                title='OptSimLUT',
                x='OpChannel',
                y='YPos',
                color='VolIdx',
                color_continuous_scale=['darkblue','red'],
                template='seaborn')
fig.update_layout(yaxis_type='linear', yaxis_title='YPos')
fig.show()

In [68]:
fig = px.scatter(df,
                title='OptSimLUT',
                x='OpChannel',
                y='Voxel',
                color=np.log(df['Visibility']*100),
                color_continuous_scale=['darkblue','red','gold'],
                template='seaborn')
fig.update_layout(yaxis_type='linear', yaxis_title='Voxel', coloraxis_colorbar_title='Visibility [%]')
fig.show()